# Query Performance Tuning

As part of this section we will go through basic performance tuning techniques with respect to queries.

* Overview of Cost Based Optimizer
* Compute Statistics for CBO
* Reading and Interpreting Explain Plans
* Criteria for creating indexes
* Criteria for Partitioning
* Writing Queries – Partition Pruning
* Overview of Query Hints

## Overview of Cost Based Optimizer

## Compute Statistics for CBO

## Reading and Interpreting Explain Plans

## Criteria for creating indexes

## Criteria for Partitioning

## Writing Queries – Partition Pruning

## Overview of Query Hints